In [125]:
import requests
import bs4
from bs4 import BeautifulSoup as bs

Here, I will scrape four different MIDI sites to get my raw MIDI data.

## http://mididb.com

In [ ]:
homepage = 'http://mididb.com'

In [ ]:
page1 = requests.get(homepage)

In [ ]:
head_soup = bs(page1.text)

In [ ]:
link_list = []
for genre in head_soup.find(id = "primary-main-menu").find_next().find_next_siblings():
    link_list.append(genre.find('a').get('href'))

In [ ]:
soup_list = []
for link in link_list:
    page = requests.get(link)
    if page.status_code == 200:
        soup_list.append(bs(page.text))
    else:
        print "Scraping error on " + link
        break

In [ ]:
assert(len(soup_list) == len(link_list))

In [ ]:
all_link_dict = {}
for soup in soup_list:
    for child in soup.find('div', class_="entry-content").find_next().find_next_sibling().find_all('a'):
        link, artist = child.get('href'), child.text
        if link not in all_link_dict:
            all_link_dict[link] = artist

In [ ]:
len(all_link_dict)

In [ ]:
for link in all_link_dict:
    if link[0] == '/':
        page = requests.get(homepage+link)
    else:
        page = requests.get(link)
    if page.status_code == 200:
        all_link_dict[link] = bs(page.text)
    else:
        print "Scraping error on " + link
        break

In [ ]:
for value in all_link_dict.values():
    assert isinstance(value, bs4.BeautifulSoup)

In [ ]:
for soup in all_link_dict.values():
    for link in soup.find_all('a'):
        url = link.get('href')
        if url[-4:] == '.mid':
            midi = requests.get(url)
            if midi.status_code == 200:
                with open('../midi/pop/' + url[url.rfind('/') + 1:], 'wb') as f:
                    f.write(midi.content)
            else:
                print "Scraping error on " + url
                break

## http://midi-archive.com

In [ ]:
homepage = 'http://midi-archive.com/'
page1 = requests.get(homepage)
page1.status_code

In [ ]:
soup = bs(page1.text)

In [ ]:
for link in soup.find_all('a'):
    url = link.get('href')
    if url and url[-4:] == '.mid':
        midi = requests.get(homepage+url)
        if midi.status_code == 200:
            with open('../midi/pop/' + url[url.rfind('/') + 1:], 'wb') as f:
                f.write(midi.content)
        else:
            print "Error scraping on " + url

## http://midiworld.com

In [ ]:
prefix = 'http://midiworld.com/download/'
for i in range(1, 4527):
    page = requests.get(prefix+str(i))
    if page.status_code == 200:
        with open('../midi/pop/midiworld_'+str(i)+'.mid', 'wb') as f:
            f.write(page.content)
    else:
        print "Error scraping on " + str(i)

## http://www.partnersinrhyme.com

In [ ]:
home = 'http://www.partnersinrhyme.com/midi/midiartists/index.shtml'

In [ ]:
page1 = requests.get(home)

In [ ]:
page1.status_code

In [ ]:
soup = bs(page1.text)

In [ ]:
PIR_links = map(lambda x: x.get('href'), soup.find_all('table', width = '471')[1].find_all('tr')[1].find_all('a'))

In [ ]:
PIR_links = set(PIR_links)

In [ ]:
soups = []
for link in PIR_links:
    page = requests.get(link)
    if page.status_code == 200:
        soups.append(bs(page.text))
    else:
        print "Error scraping on " + link

In [ ]:
len(soups)

In [ ]:
song_links = []
for soup in soups:
    for link in soup.find_all('a'):
        url = link.get('href')
        if url and url[:11] == 'midiartists':
            song_links.append('http://www.partnersinrhyme.com/midi/midiartists/'+url)

In [ ]:
len(song_links)

In [ ]:
song_soups = []
for link in song_links:
    song_page = requests.get(link)
    if song_page.status_code == 200:
        song_soups.append(bs(song_page.text))
    else:
        print "Error scraping at " + link

In [ ]:
len(song_soups)

In [ ]:
for soup in song_soups:
    link = soup.find('div', align = 'center')
    if link:
        url = link.find('a').get('href')
        if url[:4] == 'http':
            midi = requests.get(url)
        else:
            continue
        if midi.status_code == 200:
            with open('../midi/pop' + url[url.rfind('/'):], 'wb') as f:
                f.write(midi.content)
        else:
            print "Error scraping at " + url
    else:
        print "Something is off at the page " + link